In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [4]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.mini_jasper as mini_jasper
import tensorflow as tf

In [5]:
config = malaya_speech.config.ctc_featurizer_config
featurizer = malaya_speech.tf_featurization.STTFeaturizer(config)

In [6]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v[:,:,0], 0)
v, length_v

(<tf.Tensor 'ExpandDims_3:0' shape=(1, ?, 80) dtype=float32>,
 <tf.Tensor 'ExpandDims_2:0' shape=(1,) dtype=int32>)

In [7]:
model = mini_jasper.Model(v, length_v)
model.logits


Instructions for updating:
Use `tf.keras.layers.SeparableConv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


{'outputs': <tf.Tensor 'w2l_encoder/Relu_32:0' shape=(1, ?, 1024) dtype=float32>,
 'src_length': <tf.Tensor 'w2l_encoder/floordiv_64:0' shape=(1,) dtype=int32>}

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [9]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [10]:
l = sess.run(model.logits, feed_dict = {i: y})
l

{'outputs': array([[[0.        , 0.6317533 , 0.10515396, ..., 0.3214997 ,
          0.5601878 , 0.        ],
         [0.33427903, 0.        , 0.20191172, ..., 0.        ,
          0.        , 0.29395312],
         [0.01223818, 0.        , 0.        , ..., 0.42618653,
          0.        , 0.42700604],
         ...,
         [0.        , 0.20838247, 0.        , ..., 0.04816762,
          0.        , 0.22514023],
         [0.30086166, 0.04810698, 0.        , ..., 0.        ,
          0.01121054, 0.40134066],
         [0.        , 0.00588636, 0.        , ..., 0.        ,
          0.6042735 , 0.        ]]], dtype=float32),
 'src_length': array([281], dtype=int32)}

In [11]:
l = sess.run(model.logits, feed_dict = {i: y1})
l

{'outputs': array([[[1.0852382 , 0.45527968, 0.        , ..., 0.        ,
          0.        , 0.21996805],
         [0.        , 0.4513943 , 0.        , ..., 0.        ,
          0.04817728, 0.        ],
         [1.0196781 , 0.7700624 , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.05882587, 0.        , 0.        , ..., 0.        ,
          0.        , 0.05844028],
         [0.0147314 , 0.7054979 , 0.6539781 , ..., 0.11781625,
          0.        , 0.        ],
         [0.13955884, 0.4923342 , 0.        , ..., 0.        ,
          0.        , 0.41774893]]], dtype=float32),
 'src_length': array([175], dtype=int32)}